# timing_match

In [49]:
import pandas as pd # nice data utilities
import os
import numpy as np

Read in the csv file

In [50]:
fname = './data/DHO_cwv.gpx' #original gpx file
base, ext = os.path.splitext(fname)
read_filename="%s.csv" % base
gps=pd.read_table(read_filename, header=0,sep=',',names=['time','lat','lon','ele','x:speed']);

Read in the radio logging file

In [51]:
q=pd.read_table('./data/log_1.csv', header=0,sep=',',names=['t','tm','s']);

In [52]:
# get the radio times
tLog=[]
for ts in q.t:
    tLog.append(int(ts))

In [53]:
# extract arrays, just to match previous code -- can streamline later
gps.time=gps.time-18000 #GPS has 5 hour time shift

In [54]:
# now go through and compare
fl=[] # array of indices into tLog for the closest time to any given tGPS time
mins=[]
finalC=[]

for i in range(0,len(gps.time)):
    tG=gps.time[i]
    la=gps.lat[i]
    lo=gps.lon[i]
    diffs=[]
    # generate differences array
    for j in range(0,len(tLog)):
        diff=(tLog[j]-tG)**2
        diffs.append(diff)
    # find the minimum time difference
    myd=np.asarray(diffs);
    mind=min(diffs)
    # if the minimum time difference == 0, then we had a signal
    if (mind<2):
        logIndex=np.where(myd == myd.min())[0][0] #index into the log file minim time
        finalC.append((tG,la,lo,q.s[logIndex]))
    else:
        finalC.append((tG,la,lo,-1000)) # some really low signal level

In [55]:
csv_lines = []
for event in finalC:
    line = []
    #timestamp
    line.append(str(event[0])) 
    #lattitude
    line.append(str(event[1]))
    #longitude
    line.append(str(event[2]))
    #signal strength (RSSI)
    line.append(str(event[3]))
    #format and append line
    csv_lines.append(",".join(line))
    
#finish up by smashing all the lines together
csv_string = "\n".join(csv_lines)

#save the data in a suitably named file
outfile = open("%s_combined.csv" % base, 'w')
outfile.write(csv_string)
outfile.close()